# GANomaly Notebook for Individual ExperimentResults

## Initial Configurations

### Libraries import

In [1]:
import os
import sys
import cv2
import umap
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from scipy import stats
from sklearn.decomposition import PCA
from ipywidgets import interact, IntSlider
from IPython.display import display, clear_output

sys.path.append("../../")

In [2]:
from utils.metrics import accuracy, precision, recall, specificity, f1_score
from utils.metrics import dagostinoPearson_test, andersonDarling_test, shapiroWilks_test, chiSquare_test, fOneWay_test
from utils.metrics import brownForsythe_test, levene_test, bartlett_test
from utils.metrics import mannWhitney_test, kruskalWallis_test, kolmogorovSmirnov_test
from utils.savers import generate_qq_plot

### Experiment selection

In [38]:
experiment_id = "0037"
root_path = "/home/jefelitman/Saved_Models/Anomaly_parkinson/"
for i in sorted(os.listdir("/home/jefelitman/Saved_Models/Anomaly_parkinson/")):
    if experiment_id in i:
        experiment_folder = os.path.join(root_path, i)
experiment_folder

'/home/jefelitman/Saved_Models/Anomaly_parkinson/0037_Ganomaly_3D-64x64x64x1'

## Quantitative metrics

### Errors loading

In [39]:
base_path = os.path.join(experiment_folder, "outputs/errors/")
for t in ["encoder", "contextual", "adversarial"]:
        for c in ["normal", "abnormal"]:
            globals()["all_{}_{}".format(t, c)] = np.r_[[]]
            
for t in ["encoder", "contextual", "adversarial"]:
    for m in ["train", "test"]:
        classes = ["normal"] if m == "train" else ["normal", "abnormal"]
        for c in classes:
            all_data = "all_{}_{}".format(t, c)
            errors = np.load(os.path.join(base_path, t, m, c+".npy"))
            globals()["{}_{}_{}".format(m, t, c)] = errors
            globals()[all_data] = np.concatenate([globals()[all_data], errors])

### Metrics loading

In [40]:
train_metrics = pd.read_csv(os.path.join(experiment_folder, "metrics/train.csv"))
test_metrics = pd.read_csv(os.path.join(experiment_folder, "metrics/test.csv"))
print("{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}".format(
    train_metrics.loc[train_metrics.shape[0] - 1 ,"gen_error"],
    train_metrics.loc[train_metrics.shape[0] - 1 ,"disc_error"],
    test_metrics.loc[test_metrics.shape[0] - 1 ,"accuracy"],
    test_metrics.loc[test_metrics.shape[0] - 1 ,"precision"],
    test_metrics.loc[test_metrics.shape[0] - 1 ,"recall"],
    test_metrics.loc[test_metrics.shape[0] - 1 ,"specificity"],
    test_metrics.loc[test_metrics.shape[0] - 1 ,"f1_score"],
    test_metrics.loc[test_metrics.shape[0] - 1 ,"auc"]
))

0.3782039880752563	7.624619007110596	0.6339285969734192	1.0	0.5340909361839294	1.0	0.6962962962962963	0.9985795617103576


### Selecting the threshold and calculating again the metrics

In [41]:
def precision_recall_curve(y_true, y_pred, num_thresholds=200):
    precisions = []
    recalls = []
    thresholds = np.linspace(np.min(y_pred), np.max(y_pred), num_thresholds)
    for t in thresholds:
        tp = np.count_nonzero(np.logical_and(y_true, (y_pred > t)))
        fp = np.count_nonzero(np.logical_and(np.logical_not(y_true), (y_pred > t)))
        fn = np.count_nonzero(np.logical_and(y_true, (y_pred <= t)))
        if tp+fp == 0:
            precisions.append(0)
        else:
            precisions.append(precision(tp, fp))
        if tp + fn == 0:
            recalls.append(0)
        else:
            recalls.append(recall(tp, fn))
    return np.r_[precisions], np.r_[recalls], thresholds

data = "test_encoder_"
errors = ["normal", "abnormal"]
path = os.path.join(experiment_folder, "outputs/graphics/quantitative/")
y_true = np.concatenate([[i]*globals()[data + j].shape[0] for i,j in enumerate(errors)]) 
y_pred = np.concatenate([globals()[data+i] for i in errors])
y_pred = (y_pred - np.min(y_pred)) / (np.max(y_pred) - np.min(y_pred))
precisions, recalls, thresholds = precision_recall_curve(y_true, y_pred)
threshold = thresholds[np.argmin(np.abs(precisions[:-1] - recalls[:-1]))]

plt.plot(thresholds, precisions, label="Precision")
plt.plot(thresholds, recalls, label="Recall")
plt.axvline(threshold, color="black", alpha=0.5)
plt.title('Precision and Recall for different thresholds')
plt.xlabel('Threshold')
plt.ylabel('Precision/Recall')
plt.legend()
plt.savefig(os.path.join(path, "Precision_vs_Recall.png"),dpi=300)
plt.close()

TP = tf.keras.metrics.TruePositives(threshold)
TN = tf.keras.metrics.TrueNegatives(threshold)
FP = tf.keras.metrics.FalsePositives(threshold)
FN = tf.keras.metrics.FalseNegatives(threshold)

TP.update_state(y_true, y_pred)
TN.update_state(y_true, y_pred)
FP.update_state(y_true, y_pred)
FN.update_state(y_true, y_pred)

print("{}\t{}\t{}\t{}\t{}\t{}".format(
    threshold,
    np.max(accuracy(TP.result().numpy(), TN.result().numpy(), FP.result().numpy(), FN.result().numpy())),
    np.max(precision(TP.result().numpy(), FP.result().numpy())),
    np.max(recall(TP.result().numpy(), FN.result().numpy())),
    np.max(specificity(TN.result().numpy(), FP.result().numpy())),
    np.max(f1_score(TP.result().numpy(), FP.result().numpy(), FN.result().numpy()))
))

0.02512562814070352	0.9910714030265808	1.0	0.9886363744735718	1.0	0.9942857142857143


In [42]:
AUC = tf.keras.metrics.AUC()
AUC.update_state(y_true, y_pred)
AUC.result()

<tf.Tensor: shape=(), dtype=float32, numpy=0.99928975>

### Train Graphics

In [ ]:
epochs = 10000
portions = epochs // 100
columns = 5
rows = (epochs // portions) // columns
path = os.path.join(experiment_folder, "outputs/graphics/quantitative/")
for metric in ["gen_error", "disc_error", "accuracy", "specificity"]:
    fig, axs = plt.subplots(rows, columns, figsize=(30, 25))
    fig.suptitle('{} over Epochs'.format(metric))
    for i in range(rows):
        for j in range(columns):
            axs[i, j].plot(train_metrics[metric][(i*columns+j)*portions:(i*columns+j+1)*portions])
            filename = 'train_{}.png'.format(metric)
    fig.savefig(path+filename,dpi=300)
    plt.close(fig)

### Test graphics

In [ ]:
epochs = 10000
portions = epochs // 100
columns = 5
rows = (epochs // portions) // columns
path = os.path.join(experiment_folder, "outputs/graphics/quantitative/")
for metric in test_metrics.columns[1:]:
    fig, axs = plt.subplots(rows, columns, figsize=(30, 25))
    fig.suptitle('{} over Epochs'.format(metric))
    for i in range(rows):
        for j in range(columns):
            axs[i, j].plot(test_metrics[metric][(i*columns+j)*portions:(i*columns+j+1)*portions])
            filename = 'test_{}.png'.format(metric)
    fig.savefig(path+filename,dpi=300)
    plt.close(fig)

## Qualitative metrics

### Qualitative metrics

In [ ]:
for g in ["train", "test", "all"]:
    print("-------------- {} ---------------------".format(g))
    classes = ["normal"] if g == "train" else ["normal", "abnormal"]
    for c in classes:
        for t in ["encoder", "contextual", "adversarial"]:
            data = globals()["{}_{}_{}".format(g, t,c)]
            m = np.mean(data)
            s = np.std(data)
            print("{} error ({}): {}\t{}\t{}\t{}\t{}".format(
                t,
                c,
                m,
                s,
                stats.skew(data),
                stats.kurtosis(data),
                1 - stats.norm(m, s).cdf(0)
            ))
            print("")

### Normality tests

In [ ]:
for g in ["train", "test", "all"]:
    print("-------------- {} ---------------------".format(g))
    classes = ["normal"] if g == "train" else ["normal", "abnormal"]
    for c in classes:
        for t in ["encoder", "contextual", "adversarial"]:
            data = globals()["{}_{}_{}".format(g, t,c)]
            norm_dist = stats.norm.rvs(loc=np.mean(data), scale=np.std(data), size=data.shape, random_state=8128)
            chi_test = chiSquare_test(sorted(data), sorted(norm_dist))
            print("{} tests ({}): {}\t{}\t{}\t{}\t{}\t{}\t{}\t{}".format(
                t,
                c,
                int(brownForsythe_test(sorted(data), sorted(norm_dist))),
                int(levene_test(sorted(data), sorted(norm_dist))),
                int(bartlett_test(sorted(data), sorted(norm_dist))),
                int(dagostinoPearson_test(data)),
                int(andersonDarling_test(data)),
                int(shapiroWilks_test(data)),
                "{} ({})".format(int(chi_test[0]), round(chi_test[1], 5)),
                int(fOneWay_test(sorted(data), sorted(norm_dist)))
            ))
            print("")

### Grouping tests

In [ ]:
for t in ["encoder", "contextual", "adversarial"]:
    print("-------------- {} ---------------------".format(t))
    for g1, g2 in [("train", "test"), ("test", "all"), ("train", "all")]:
        data1 = sorted(globals()["{}_{}_normal".format(g1, t)])
        data2 = sorted(globals()["{}_{}_normal".format(g2, t)])
        print("{} vs {}: {}\t{}\t{}\t{}\t{}\t{}\t{}".format(
            g1,
            g2,
            int(brownForsythe_test(data1, data2)),
            int(levene_test(data1, data2)),
            int(bartlett_test(data1, data2)),
            int(mannWhitney_test(data1, data2)),
            int(kruskalWallis_test(data1, data2)),
            int(kolmogorovSmirnov_test(data1, data2)),
            int(fOneWay_test(data1, data2))
        ))
        print("")

### Classing tests

In [ ]:
for g in ["test", "all"]:
    print("-------------- {} ---------------------".format(g))
    for t in ["encoder", "contextual", "adversarial"]:
        data1 = sorted(globals()["{}_{}_normal".format(g, t)])
        data2 = sorted(globals()["{}_{}_abnormal".format(g, t)])
        line = "{} normal vs abnormal: {}\t{}\t{}".format(
            t,
            int(brownForsythe_test(data1, data2)),
            int(levene_test(data1, data2)),
            int(bartlett_test(data1, data2)),
        )
        if g == "test":
            line += "\t{}\t{}\t{}\t{}".format(
                int(mannWhitney_test(data1, data2)),
                int(kruskalWallis_test(data1, data2)),
                int(kolmogorovSmirnov_test(data1, data2)),
                int(fOneWay_test(data1, data2))
            )
        else:
            chi_1_test = chiSquare_test(data1, data2)
            chi_2_test = chiSquare_test(data2, data1)
            line += "\t{}\t{}\t{}\t{}\t{}\t{}".format(
                "{} ({})".format(int(chi_1_test[0]), round(chi_1_test[1], 5)),
                "{} ({})".format(int(chi_2_test[0]), round(chi_2_test[1], 5)),
                int(mannWhitney_test(data1, data2)),
                int(kruskalWallis_test(data1, data2)),
                int(kolmogorovSmirnov_test(data1, data2)),
                int(fOneWay_test(data1, data2))
            )
        print(line)
        print("")

### Train graphics

#### QQ plots

In [ ]:
save_path = os.path.join(experiment_folder, "outputs/graphics/qualitative/")
for t in ["encoder", "contextual", "adversarial"]:
    for c in ["normal"]:
        data = globals()["train_{}_{}".format(t,c)]
        generate_qq_plot(
            data, 
            save_path, 
            "train_{}_{}".format(t,c),
            ".png",
            np.mean(data),
            np.std(data)
        )

### Test graphics

#### Box plots

In [ ]:
save_path = os.path.join(experiment_folder, "outputs/graphics/qualitative/")
rows = 1
columns = 3
fig, axs = plt.subplots(rows, columns, figsize=(15, 10))
for i, t in enumerate(["encoder", "contextual", "adversarial"]):
    data = "test_{}_".format(t)
    axs[i].boxplot([globals()[data+"normal"], globals()[data+"abnormal"]], labels=['Normal', 'Abnormal'])
    axs[i].set_title("{} errors".format(t))

filename = 'test_boxplot.png'
fig.savefig(save_path+filename)
plt.close(fig)

#### QQ plots

In [ ]:
save_path = os.path.join(experiment_folder, "outputs/graphics/qualitative/")
for t in ["encoder", "contextual", "adversarial"]:
    for c in ["normal", "abnormal"]:
        data = globals()["test_{}_{}".format(t,c)]
        generate_qq_plot(
            data, 
            save_path, 
            "test_{}_{}".format(t,c),
            ".png",
            np.mean(data),
            np.std(data)
        )

### All data graphics

#### Box plots

In [ ]:
save_path = os.path.join(experiment_folder, "outputs/graphics/qualitative/")
rows = 1
columns = 3
fig, axs = plt.subplots(rows, columns, figsize=(15, 10))
for i, t in enumerate(["encoder", "contextual", "adversarial"]):
    data = "all_{}_".format(t)
    axs[i].boxplot([globals()[data+"normal"], globals()[data+"abnormal"]], labels=['Normal', 'Abnormal'])
    axs[i].set_title("{} errors".format(t))

filename = 'all_data_boxplot.png'
fig.savefig(save_path+filename)
plt.close(fig)

#### QQ plots

In [ ]:
save_path = os.path.join(experiment_folder, "outputs/graphics/qualitative/")
for t in ["encoder", "contextual", "adversarial"]:
    for c in ["normal", "abnormal"]:
        data = globals()["all_{}_{}".format(t,c)]
        generate_qq_plot(
            data, 
            save_path, 
            "all_{}_{}".format(t,c),
            ".png",
            np.mean(data),
            np.std(data)
        )

## Visual results

### Loading latent vectors

In [ ]:
base_path = os.path.join(experiment_folder, "outputs/latent_vectors/")
for n in ["generator", "discriminator"]:
    for t in ["input", "output"]:
        for c in ["normal", "abnormal"]:
            globals()["all_{}_{}_{}".format(n, t, c)] = []
            
for n in ["generator", "discriminator"]:
    for t in ["input", "output"]:
        for m in ["train", "test"]:
            classes = ["normal"] if m == "train" else ["normal", "abnormal"]
            for c in classes:
                all_data = "all_{}_{}_{}".format(n, t, c)
                data = "{}_{}_{}_{}".format(m, n, t, c)
                globals()[data] = []
                path = os.path.join(base_path, t + "_" + n, m, c)
                for file in sorted(os.listdir(path)):
                    vector = np.load(os.path.join(path, file))
                    globals()[data].append(vector)
                    globals()[all_data].append(vector)
                globals()[data] = np.r_[globals()[data]]

for n in ["generator", "discriminator"]:
    for t in ["input", "output"]:
        for c in ["normal", "abnormal"]:
            all_data = "all_{}_{}_{}".format(n, t, c)
            globals()[all_data] = np.r_[globals()[all_data]]
            
plt.rc("text", usetex=True)

### UMAP latent vectors

In [ ]:
save_path = os.path.join(experiment_folder, "outputs/graphics/visuals/")
for n in ["generator", "discriminator"]:
    for t in ["input", "output"]:
        for m in ["train", "test", "all"]:
            classes = ["normal"] if m == "train" else ["normal", "abnormal"]
            globals()["{}_mapped_{}_{}".format(m, n, t)] = umap.UMAP(random_state = 8128).fit_transform(np.concatenate(
                    [globals()["{}_{}_{}_{}".format(m, n, t, c)] for c in classes], axis=0
                )
            )

            divisor = globals()["{}_{}_{}_normal".format(m, n, t)].shape[0]
            plt.scatter(globals()["{}_mapped_{}_{}".format(m, n, t)][:divisor,0],
                globals()["{}_mapped_{}_{}".format(m, n, t)][:divisor,1], color="blue", s=5, label=r"Control $Z_{G}$"
            )
            if "abnormal" in classes:
                plt.scatter(globals()["{}_mapped_{}_{}".format(m, n, t)][divisor:,0],
                    globals()["{}_mapped_{}_{}".format(m, n, t)][divisor:,1], color="red", s=5, label=r"Parkinson $Z_{G}$"
                )
            plt.legend()
            plt.title("UMAP for {} {} data in {}".format(n, t, m))
            filename = '{}_umap_{}_{}.png'.format(m, n, t)
            plt.savefig(save_path+filename,dpi=300)
            plt.close()

### UMAP combined latent vectors

In [ ]:
save_path = os.path.join(experiment_folder, "outputs/graphics/visuals/")
for n in ["generator", "discriminator"]:
    for m in ["train", "test", "all"]:
        classes = ["abnormal"] if m == "train" else ["abnormal", "normal"]
        for c in classes:
            input_data = globals()["{}_{}_input_{}".format(m, n, c)]
            output_data = globals()["{}_{}_output_{}".format(m, n, c)]
            globals()["{}_{}_combined_{}".format(m, n, c)] = np.concatenate([input_data, output_data])
            
        globals()["{}_mapped_{}_combined".format(m, n)] = umap.UMAP(random_state = 8128).fit_transform(np.concatenate(
                [globals()["{}_{}_combined_{}".format(m, n, c)] for c in classes], axis=0
            )
        )
        divisor_normal = globals()["{}_{}_input_abnormal".format(m, n)].shape[0]
        divisor_classes = globals()["{}_{}_combined_abnormal".format(m, n)].shape[0]
        plt.scatter(globals()["{}_mapped_{}_combined".format(m, n)][:divisor_normal,0],
            globals()["{}_mapped_{}_combined".format(m, n)][:divisor_normal,1], 
            color="blue", 
            s=20, 
            label=r"Parkinson $Z_{G}$",
            marker="|"
        )
        plt.scatter(globals()["{}_mapped_{}_combined".format(m, n)][divisor_normal:divisor_classes,0],
            globals()["{}_mapped_{}_combined".format(m, n)][divisor_normal:divisor_classes,1], 
            color="cyan", 
            s=20, 
            label=r"Parkinson $Z'_{G}$",
            marker="_"
        )
        if "normal" in classes:
            divisor_abnormal = globals()["{}_{}_input_normal".format(m, n)].shape[0] + divisor_classes
            plt.scatter(globals()["{}_mapped_{}_combined".format(m, n)][divisor_classes:divisor_abnormal,0],
                globals()["{}_mapped_{}_combined".format(m, n)][divisor_classes:divisor_abnormal,1], 
                color="red", 
                s=20, 
                label=r"Normal $Z_G$",
                marker="|"
            )
            plt.scatter(globals()["{}_mapped_{}_combined".format(m, n)][divisor_abnormal:,0],
                globals()["{}_mapped_{}_combined".format(m, n)][divisor_abnormal:,1], 
                color="orange", 
                s=20, 
                label=r"Normal $Z'_G$",
                marker="_"
            )
        plt.legend()
        plt.title("UMAP for {} combined data in {}".format(n, m))
        filename = '{}_umap_{}_combined.png'.format(m, n)
        plt.savefig(save_path+filename,dpi=300)
        plt.close()

### PCA latent vectors

In [ ]:
save_path = os.path.join(experiment_folder, "outputs/graphics/visuals/")
for n in ["generator", "discriminator"]:
    for t in ["input", "output"]:
        for m in ["train", "test", "all"]:
            classes = ["normal"] if m == "train" else ["normal", "abnormal"]
            globals()["{}_mapped_{}_{}".format(m, n, t)] = PCA(n_components=2, random_state=8128).fit_transform(np.concatenate(
                    [globals()["{}_{}_{}_{}".format(m, n, t, c)] for c in classes], axis=0
                )
            )

            divisor = globals()["{}_{}_{}_normal".format(m, n, t)].shape[0]
            plt.scatter(globals()["{}_mapped_{}_{}".format(m, n, t)][:divisor,0],
                globals()["{}_mapped_{}_{}".format(m, n, t)][:divisor,1], color="blue", s=5, label=r"Control $Z_{G}$"
            )
            if "abnormal" in classes:
                plt.scatter(globals()["{}_mapped_{}_{}".format(m, n, t)][divisor:,0],
                    globals()["{}_mapped_{}_{}".format(m, n, t)][divisor:,1], color="red", s=5, label=r"Parkinson $Z_{G}$"
                )
            plt.legend()
            plt.title("PCA for {} {} data in {}".format(n, t, m))
            filename = '{}_pca_{}_{}.png'.format(m, n, t)
            plt.savefig(save_path+filename,dpi=300)
            plt.close()

### PCA combined latent vectors

In [ ]:
save_path = os.path.join(experiment_folder, "outputs/graphics/visuals/")
for n in ["generator", "discriminator"]:
    for m in ["train", "test", "all"]:
        classes = ["normal"] if m == "train" else ["normal", "abnormal"]
        for c in classes:
            input_data = globals()["{}_{}_input_{}".format(m, n, c)]
            output_data = globals()["{}_{}_output_{}".format(m, n, c)]
            globals()["{}_{}_combined_{}".format(m, n, c)] = np.concatenate([input_data, output_data])
            
        globals()["{}_mapped_{}_combined".format(m, n)] = PCA(n_components=2, random_state=8128).fit_transform(np.concatenate(
                [globals()["{}_{}_combined_{}".format(m, n, c)] for c in classes], axis=0
            )
        )
        divisor_normal = globals()["{}_{}_input_normal".format(m, n)].shape[0]
        divisor_classes = globals()["{}_{}_combined_normal".format(m, n)].shape[0]
        plt.scatter(globals()["{}_mapped_{}_combined".format(m, n)][:divisor_normal,0],
            globals()["{}_mapped_{}_combined".format(m, n)][:divisor_normal,1], 
            color="blue", 
            s=20, 
            label=r"Control $Z_{G}$",
            marker="|"
        )
        plt.scatter(globals()["{}_mapped_{}_combined".format(m, n)][divisor_normal:divisor_classes,0],
            globals()["{}_mapped_{}_combined".format(m, n)][divisor_normal:divisor_classes,1], 
            color="cyan", 
            s=20, 
            label=r"Control $Z'_{G}$",
            marker="_"
        )
        if "abnormal" in classes:
            divisor_abnormal = globals()["{}_{}_input_abnormal".format(m, n)].shape[0] + divisor_classes
            plt.scatter(globals()["{}_mapped_{}_combined".format(m, n)][divisor_classes:divisor_abnormal,0],
                globals()["{}_mapped_{}_combined".format(m, n)][divisor_classes:divisor_abnormal,1], 
                color="red", 
                s=20, 
                label=r"Parkinson $Z_G$",
                marker="|"
            )
            plt.scatter(globals()["{}_mapped_{}_combined".format(m, n)][divisor_abnormal:,0],
                globals()["{}_mapped_{}_combined".format(m, n)][divisor_abnormal:,1], 
                color="orange", 
                s=20, 
                label=r"Parkinson $Z'_G$",
                marker="_"
            )
        plt.legend()
        plt.title("PCA for {} combined data in {}".format(n, m))
        filename = '{}_pca_{}_combined.png'.format(m, n)
        plt.savefig(save_path+filename,dpi=300)
        plt.close()

### Video comparision

In [ ]:
def show_all_videos(real_videos, fake_videos, substraction_videos):
    assert len(real_videos) == len(fake_videos) == len(substraction_videos)
    for i in range(len(real_videos)):
        assert real_videos[i].shape == fake_videos[i].shape == substraction_videos[i].shape
    
    frame_slider = IntSlider(min=1, max=real_videos[0].shape[0], step=1)
    volume_slider = IntSlider(min=1, max=len(real_videos), step=1)

    def update_frame_max(*args):
        frame_slider.max = real_videos[volume_slider.value].shape[0]
    volume_slider.observe(update_frame_max, 'value')
    
    interact(lambda volume, frame: plt.imshow(real_videos[volume-1][frame-1]),
        volume=volume_slider, frame=frame_slider
    )
    interact(lambda volume, frame: plt.imshow(fake_videos[volume-1][frame-1]),
        volume=volume_slider, frame=frame_slider
    )
    interact(lambda volume, frame: plt.imshow(substraction_videos[volume-1][frame-1]),
        volume=volume_slider, frame=frame_slider
    )

In [ ]:
base_path = os.path.join(experiment_folder, "outputs/samples/")
for t in ["real", "fake", "substraction"]:
    for c in ["normal", "abnormal"]:
        globals()["{}_{}_videos".format(t, c)] = []

for t in ["real", "fake", "substraction"]:
    for m in ["train", "test"]:
        classes = ["normal"] if m == "train" else ["normal", "abnormal"]
        for c in classes:
            videos_path = os.path.join(base_path, t, m, c)
            for video_folder in sorted(os.listdir(videos_path)):
                video = []
                video_path = os.path.join(videos_path, video_folder)
                for frame in sorted(os.listdir(video_path)):
                    video.append(cv2.cvtColor(
                        cv2.imread(
                            os.path.join(video_path, frame)
                        ), cv2.COLOR_BGR2RGB
                    ))
                globals()["{}_{}_videos".format(t, c)].append(np.r_[video])
len(real_normal_videos)

In [ ]:
videos = "normal"
show_all_videos(
    globals()["real_{}_videos".format(videos)],
    globals()["fake_{}_videos".format(videos)],
    globals()["substraction_{}_videos".format(videos)],
)